# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [25]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [26]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):

  dfda = -3 * a**2 - 0.5 * a ** -0.5
  dfdb = 3 * cos(3*b) + 2.5 * b ** 1.5
  dfdc = 1 / c**2
  return [dfda, dfdb, dfdc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [27]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------

h = 0.000001
a1, b1, c1 = 2, 3, 4

y1 = f(a1, b1, c1)

dfda = (f(a1 + h, b1, c1) - y1) / h
dfdb = (f(a1, b1 + h, c1) - y1) / h
dfdc = (f(a1, b1, c1 + h) - y1) / h

numerical_grad = [dfda, dfdb, dfdc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [28]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------

y1 = f(a1, b1, c1)

dfda = (f(a1 + h, b1, c1) - f(a1 - h, b1, c1)) / (2 * h)
dfdb = (f(a1, b1 + h, c1) - f(a1, b1 - h, c1)) / (2 * h)
dfdc = (f(a1, b1, c1 + h) - f(a1, b1, c1 - h)) / (2 * h)
numerical_grad2 = [dfda, dfdb, dfdc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [29]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  def __radd__(self, other):
    return self.__add__(other)

  def exp(self):
    e = exp(self.data)
    out = Value(e, (self,), 'e^')
    def _backward():
      self.grad += e * out.grad

    out._backward = _backward
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    
    return out

  def __neg__(self):
    return self * -1

  def __pow__(self, k):
    if isinstance(k, Value):
      raise 'pow only supported for scalars for now'

    out = Value(self.data ** k, (self, ), '**')
    def _backward():
      self.grad += k * self.data**(k-1) * out.grad
    
    out._backward = _backward
    return out


  def __truediv__(self, other):
    return self * other ** -1

  def log(self):
    l = log(self.data)
    out = Value(l, (self,), 'log')
    def _backward():
      self.grad += out.grad / self.data
    
    out._backward = _backward
    return out

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [30]:

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out


### testing the forward pass for newly written functions

In [31]:
import numpy as np

# mul

a = Value(2.0)
b = Value(-3.0)

c = a * b
assert c.data == -6.0


# exp

e1 = a.exp()
assert exp(a.data) == e1.data

# pow

p1 = a ** 2

assert p1.data == 4

# log

l1 = a.log()
np.testing.assert_almost_equal(l1.data, log(a.data))

# negation
n1 = -a
assert n1.data == -2
# true division

### testing the backward pass for newly written functions

In [32]:
# confirm that softmax backward pass gives the
# same gradient as torch

logits = [Value(10.0), Value(3.0)]
probs = softmax(logits)
probs[1].backward()
print(f"dim 0 grad {logits[0].grad}")
print(f"dim 1 grad {logits[1].grad}")


#### pytorch version
import torch
logitst = torch.Tensor([10, 3]); logitst.requires_grad = True
probs = torch.softmax(logitst, dim=0)
print(probs[1])
probs[1].backward()
print(logitst.grad.data)

dim 0 grad -0.0009102211801218266
dim 1 grad 0.0009102211801218265
tensor(0.0009, grad_fn=<SelectBackward0>)
tensor([-0.0009,  0.0009])


In [33]:
# verify that the log backward pass gives the
# same gradient as torch

a = Value(2.0)
b = -a.log()

b.backward()
print(f"a grad {a.grad}")

#### pytorch version
import torch
at = torch.Tensor([2.0]); at.requires_grad=True
bt = -at.log()

bt.backward()
print(f"a grad", at.grad.data)

a grad -0.5
a grad tensor([-0.5000])


### test softmax + NLL

In [34]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [35]:
# verify the softmax + NLL results match pytorch
test_inputs = [10.0, 3.0, -1.0, 7.0]
label_idx = 2

logits = [Value(x) for x in test_inputs]
probs = softmax(logits)
print(probs)
loss = -probs[label_idx].log()
loss.backward()

grads = [l.grad for l in logits]

#### pytorch version

logitst = torch.Tensor(test_inputs); logitst.requires_grad = True
probs = torch.softmax(logitst, dim=0)
loss = -probs[label_idx].log()
loss.backward()

gradst = logitst.grad.data.tolist()

np.testing.assert_array_almost_equal(grads, gradst)


[Value(data=0.9517322770094628), Value(data=0.0008678674994410643), Value(data=1.5895547723031246e-05), Value(data=0.047383959943372946)]
